In [6]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import requests
import urllib

import cv2
import matplotlib.pyplot as plt

import math
import numpy as np
from scipy.ndimage import imread

import sys
import time
import os
from copy import deepcopy
import ctypes

In [7]:
HWC = (360, 480, 3)
WH = (480, 360)
IP = "13.125.182.116"
DB_SERVER_IP = "http://" + IP + ":8090/auth/login"
YML_IP = "http://" + IP + ":8090/face-model"
LECTURE_IP = "http://" + IP + ":8090/face-model/lectures"
ATTENDANCE_IP =  "http://" + IP + ":8090/face-model/attendance"
HEARTBEAT_IP = "http://" + IP + ":8090/face-model/lecture_check"

###############################################################################################################
class BeforeLogin(QWidget):
    def __init__(self, path=['haarcascades/haarcascade_frontalface_default.xml', 'haarcascades/haarcascade_eye.xml']):
        QWindow.__init__(self)
        self.path = path
        self.token = False
        
        self.central_layout = QGridLayout()
        self.central_widget = QStackedWidget()
        self.central_layout.addWidget(self.central_widget)
        
        lst = ["ID  : ", "PW : "]
        self.label_lst, self.edit_lst = self.making_Label_Edit(lst)
        pos_lst = [(1,1), (2,1)]
        
        layout = QGridLayout() 
        LoginFrame = self.making_Frame(layout, self.label_lst, self.edit_lst, pos_lst)
        LoginFrame.setFrameShape(QFrame.StyledPanel)
        
        LoginButton = QPushButton("로그인 하기")
        LoginButton.clicked.connect(self.btnOkClicked)
        
        LoginLayout = QGridLayout() 
        LoginLayout.addWidget(LoginFrame)
        LoginLayout.addWidget(LoginButton)
        
        coveringWidget = QWidget()
        coveringWidget.setLayout(LoginLayout)
        self.central_widget.addWidget(coveringWidget)
        self.setLayout(self.central_layout)
        
        print("#"*60)
        print("[{}] : program start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
        
    def btnOkClicked(self):
        ID = self.edit_lst[0].text().strip()
        PW = self.edit_lst[1].text().strip()
        
        self.login(ID, PW)

    def making_Label_Edit(self, name_lst) :
        label_lst = []
        edit_lst = []
        
        for name in name_lst :
            label = QLabel(name)
            edit = QLineEdit()
            
            label_lst.append(label)
            edit_lst.append(edit)
        
        return label_lst, edit_lst

    def making_Frame(self, layout, label_lst, widget_lst, pos_lst) :
        for label, widget, pos in zip(label_lst, widget_lst, pos_lst) :  
            layout.addWidget(label, pos[0], pos[1])
            layout.addWidget(widget, pos[0], pos[1]+1)
            
        frame = QFrame()
        frame.setLayout(layout)
        
        return frame
    
    def login(self, ID, PW) :
        print("#"*60)
        print("[{}] : '{}' try to login".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), ID))
        
        if self.checkUserDB(ID,PW) :
            print("[{}] : '{}' success to login".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), ID))
            print("#"*60, "\n")
            
            # server로 부터 model을 가져옴
            self.isModel = [0]
            self.wThread = WaitingThread(self.token, self.isModel)
            self.wThread.start()
            
            # Lecture list 가져오기
            header = {"x-access-token" : self.token}
            result = requests.get(LECTURE_IP, headers= header)
            
            # Lecture 선택화면으로 이동
            self.lectureWidget = LectureWidget(result.json(), self.isModel)
            self.lectureWidget.setToken(self.token)
            self.lectureWidget.setCentralWidget(self.central_widget)
            self.central_widget.addWidget(self.lectureWidget)
            self.central_widget.setCurrentWidget(self.lectureWidget)

            
        else :
            print("[{}] : '{}' fail to login".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), ID))
            print("#"*60, "\n")
            
            QMessageBox.information(self, "Login failed", "check ID & PW")
        
    def AfterLogin(self) :
        self.afterLoginWidget = AfterLoginWidget()
        return self.afterLoginWidget
    
    # 로그인 시도, 성공하면 token을 가져옴
    def checkUserDB(self, ID, PW, host=DB_SERVER_IP) :
        data = {"email_id" : ID, "password" : PW}
        result = requests.post(host, data=data)
        
        if "success" in result.json() :
            self.token = result.json()["token"]
            
            with open("./data/register/registered_info/token.txt", "w") as f:
                f.write(self.token)
            
            return True
        else :
            return False


class WaitingThread(QThread) :
    def __init__(self, token, isModel):
        QThread.__init__(self)
        self.token = token
        self.isModel = isModel
        
    def __del__(self):
        self.wait()

    def run(self):
        self.loading()

    def loading(self) :
        print("#"*60)
        print("[{}] : loading token".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        
        # 모델 가져오기
        header = {"x-access-token" : self.token}
        result = requests.get(YML_IP, headers= header)
        
        # 모델이 있는 경우
        if result.json()["model"] :
            print("Model is found")
            url = result.json()["model"]
            urllib.request.urlretrieve(url, './data/register/registered_info/train1.yml') 
            self.isModel[0] = 1
            
            print("Model is loaded")

        # 모델이 없는 경우
        else :
            self.isModel[0] = 2
            print("Model is not found")
        print("#"*60, "\n")
###############################################################################################################



###############################################################################################################
class LectureWidget(QWidget):
    def __init__(self, l_json, isModel, parent=None):
        super().__init__(parent)
        self.isModel = isModel
        
        layout = QVBoxLayout()
        
        self.button_lst = []
        for lecture in l_json :
            button = QPushButton(lecture["lecture_name"] + "/" + lecture["teacher_name"])
            self.button_lst.append((button, lecture["lecture_id"], lecture["is_auto"]))
            
        for button_info in self.button_lst :
            button_info[0].clicked.connect(lambda: self.lectureClicked(button_info[1], button_info[2]))
            layout.addWidget(button_info[0])
            
        self.setLayout(layout)
        
    def setToken(self, token) :
        self.token = token
        
    def setCentralWidget(self, widget) :
        self.central_widget = widget
        
    def lectureClicked(self, lecture_id, ongoing) :
        # 강의 진행 중
        if ongoing == 1 :
            self.afterLoginWidget = AfterLoginWidget(lecture_id, self.isModel, self.token)
            self.central_widget.addWidget(self.afterLoginWidget)
            self.central_widget.setCurrentWidget(self.afterLoginWidget)
            
            print("[{}] : {} is lecture time".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), lecture_id))
            print("#"*60, "\n")
        # 강의 진행 중이 아님
        else :
            QMessageBox.information(self, "Access failed", "Not Lecture Time")
            print("[{}] : {} is not lecture time".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time())), lecture_id))
            print("#"*60, "\n")
###############################################################################################################



###############################################################################################################
class AfterLoginWidget(QWidget):
    def __init__(self, lecture_id, isModel, token, path=['haarcascades/haarcascade_frontalface_default.xml', 'haarcascades/haarcascade_eye.xml'], interval=3, parent=None):
        super().__init__(parent)
        
        # global_flag : dialog가 실행되었는지 안되었는지 확인하기 위한 변수
        #               True = 안열려있음, False = 열려있음
        self.lecture_id = lecture_id
        self.isModel = isModel
        self.token = token
        self.interval = interval
        self.path = path
        self.global_flag = [True] 
        self.dialog_size = (480,360)

        layout = QVBoxLayout()
        self.ready_detection()

        self.RegisterButton = QPushButton('Register')
        self.RunButton = QPushButton('Attendance Check')

        self.RegisterButton.clicked.connect(self.registerClicked)
        self.RunButton.clicked.connect(self.runClicked)

        layout.addWidget(self.RegisterButton)
        layout.addWidget(self.RunButton)
        self.setLayout(layout)

        self.heartbeatThread = HeartbeatThread(self.lecture_id, self.token)
        self.heartbeatThread.setRunButton(self.RunButton)
        self.heartbeatThread.start()
        
    def registerClicked(self) :
        # TODO : register 허가 요청하기 
        
        # model이 로딩 중일 경우
        if self.isModel[0] == 0 :
            QMessageBox.information(self, "Register failed", "Waiting for loading model")
            
        else :
            # 다른 dilaog가 안열려 있을 경우
            if self.global_flag[0] :
                print("#"*60)
                print("[{}] : try to register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                print("#"*60, "\n")

                self.global_flag[0] = False
                params = [self.video_capture, self.face_cascade, self.eyes_cascade, self.interval, self.global_flag, self.lecture_id]
                self.registerDialog = registerDialog(params, self.dialog_size, self, self.token)
                self.registerDialog.show()

            # 다른 dilaog가 열려 있을 경우
            else : 
                print("#"*60)
                print("[{}] : stop the other task first".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                print("#"*60, "\n")
                QMessageBox.information(self, "Register failed", "stop the other task first")
        
    def runClicked(self) :
        # 출석체크가 가능한 경우 확인
        header = {"x-access-token" : self.token}
        result = requests.get(HEARTBEAT_IP+"/"+str(self.lecture_id), headers = header)
        if "check" in result.json() :
            if not result.json()["check"] :
                QMessageBox.information(self, "Attendance check failed", "permission is not allowed")
                return
        else :
            QMessageBox.information(self, "Attendance check failed", "permission is not allowed")
            return
        
        # model이 로딩 중일 경우
        if self.isModel[0] == 0 :
            QMessageBox.information(self, "Attendance check failed", "waiting for loading model")
            return
        
        # model이 없을 경우
        elif self.isModel[0] == 2 :
            QMessageBox.information(self, "Attendance check failed", "register first")
            return

        else :
            # 다른 dilaog가 안열려 있을 경우
            if self.global_flag[0] :
                print("#"*60)
                print("[{}] : try to attendance check".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                print("#"*60, "\n")

                self.global_flag[0] = False
                params = [self.video_capture, self.face_cascade, self.eyes_cascade, self.interval, self.global_flag, self.lecture_id]
                self.attendanceDialog = attendanceDialog(params, self.dialog_size, self, self.token)
                self.attendanceDialog.setHeartbeatThread(self.heartbeatThread)
                self.attendanceDialog.show()

            # 다른 dilaog가 열려 있을 경우
            else : 
                print("#"*60)
                print("[{}] : stop the other task first".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                print("#"*60, "\n")
                QMessageBox.information(self, "Register failed", "stop the other task first")
 
    def ready_detection(self) : 
        self.face_cascade = cv2.CascadeClassifier(self.path[0])
        self.eyes_cascade = cv2.CascadeClassifier(self.path[1])
        
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
            
        self.video_capture = cv2.VideoCapture(0)
        _, first_frame = self.video_capture.read()

        print("#"*60)
        try : 
            if not first_frame :
                print("[{}] : camera is not conntected".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                QMessageBox.information(self, "Error", "Camera is not connected")
        except :
            print("[{}] : camera is conntected".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            
            
class HeartbeatThread(QThread) :
    def __init__(self, lecture_id, token, interval=10):
        QThread.__init__(self)
        self.lecture_id = lecture_id
        self.token = token
        self.interval = interval
        self.state = True
        self.startCheck = False
        
    def __del__(self):
        self.state = False
        self.wait()

    def run(self):
        self.heartbeat()
        
    def setRunButton(self, button) :
        self.runButton = button
    
    def heartbeat(self) :
        while self.state :
            time.sleep(self.interval)
            header = {"x-access-token" : self.token}
            result = requests.get(HEARTBEAT_IP+"/"+str(self.lecture_id), headers=header)

            if "check" in result.json() :
                if result.json()["check"] and not self.startCheck :
                    self.runButton.click()
                    self.startCheck = True
                    
                
###############################################################################################################



###############################################################################################################
class registerDialog(QDialog):
    def __init__(self, params, size, widget, token) :
        QDialog.__init__(self)  
        self.params = params
        self.setParams(params)
        
        self.setWindowTitle('Register')
        self.resize(size[0], size[1])
        self.widget = widget
        self.token = token
        self.start = False
        
        self.registerThread = registerThread(self.lecture_id)
        self.registerThread.setDialog(self)
        self.registerThread.setToken(self.token)

        self.msg = QLabel()
        self.imageLabel = QLabel()
        self.RunButton = QPushButton('Start')
        self.ExitButton = QPushButton('Stop')

        self.RunButton.clicked.connect(self.runClicked)
        self.ExitButton.clicked.connect(self.stopClicked)

        self.layout = QVBoxLayout()
        self.layout.addWidget(self.msg)
        self.layout.addWidget(self.imageLabel)
        self.layout.addWidget(self.RunButton)
        self.layout.addWidget(self.ExitButton)
        
        self.registerThread.setImageLabel(self.imageLabel)
        self.registerThread.setParams(params)
        self.setLayout(self.layout)
        
        self.registerThread.start()
        
    def setParams(self, params) : 
        self.video_capture = params[0]
        self.face_cascade = params[1]
        self.eyes_cascade = params[2]
        self.interval = params[3]
        self.global_flag = params[4]
        self.lecture_id = params[5]
        
    def runClicked(self) :
        if not self.start :
            print("#"*60)
            print("[{}] : start register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            print("#"*60, "\n")
            print("[{}] : register - step1 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))

            self.registerThread.start_register = True
            self.registerThread.start_saving = False
            self.registerThread.finish_saving = [False]
            self.registerThread.detected_lst = []
            self.registerThread.coord_lst = []
            self.registerThread.confidence_lst = []
            self.registerThread.eye_count = 0
            self.start = True
        
    def stopClicked(self) :
        if self.start : 
            print("#"*60)
            print("[{}] : stop register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            print("#"*60, "\n")

            self.registerThread.start_register = False
            self.registerThread.start_saving = False
            self.registerThread.finish_saving = [False]
            self.registerThread.detected_lst = []
            self.registerThread.coord_lst = []
            self.registerThread.confidence_lst = []
            self.registerThread.eye_count = 0
            self.start = False
            self.global_flag[0] = True
            
    def closeEvent(self, event):
        self.registerThread.go_flag = False
        self.registerThread.quit()
        self.global_flag[0] = True
        
        print("#"*60)
        print("[{}] : close register".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
                
        event.accept()
###############################################################################################################


###############################################################################################################
class attendanceDialog(QDialog):
    def __init__(self, params, size, widget, token) :
        QDialog.__init__(self)  
        self.params = params
        self.setParams(params)
        
        self.setWindowTitle('Attendance')
        self.resize(size[0], size[1])
        self.widget = widget
        self.token =token
        self.start = False
        
        self.attendanceThread = attendanceThread(self.lecture_id, self.token)
        self.attendanceThread.setParams(params)
        self.attendanceThread.setDialog(self)

        self.msg = QLabel()
        self.imageLabel = QLabel()
        self.RunButton = QPushButton('Start')
        self.ExitButton = QPushButton('Stop')

        self.RunButton.clicked.connect(self.runClicked)
        self.ExitButton.clicked.connect(self.stopClicked)

        self.layout = QVBoxLayout()
        self.layout.addWidget(self.msg)
        self.layout.addWidget(self.imageLabel)
        self.layout.addWidget(self.RunButton)
        self.layout.addWidget(self.ExitButton)
        
        self.attendanceThread.setImageLabel(self.imageLabel)
        self.attendanceThread.setParams(params)
        self.setLayout(self.layout)
        
        self.attendanceThread.start()
        
    def setParams(self, params) : 
        self.video_capture = params[0]
        self.face_cascade = params[1]
        self.eyes_cascade = params[2]
        self.interval = params[3]
        self.global_flag = params[4]
        self.lecture_id = params[5]
        
    def setHeartbeatThread(self, heartbeatThread) :
        self.heartbeatThread = heartbeatThread
        
    def runClicked(self) :
        if not self.start :
            print("#"*60)
            print("[{}] : start attendance check".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            print("#"*60, "\n")
            print("[{}] : attendance check - step1 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))

            self.attendanceThread.start_register = True
            self.attendanceThread.start_saving = False
            self.attendanceThread.finish_saving = [False]
            self.attendanceThread.detected_lst = []
            self.attendanceThread.coord_lst = []
            self.attendanceThread.confidence_lst = []
            self.start = True
            
    def stopClicked(self) :
        if self.start : 
            print("#"*60)
            print("[{}] : stop attendance check".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
            print("#"*60, "\n")

            self.attendanceThread.start_register = False
            self.attendanceThread.start_saving = False
            self.attendanceThread.finish_saving = [False]
            self.attendanceThread.coord_lst = []
            self.attendanceThread.confidence_lst = []
            self.start = False
            self.global_flag[0] = True
            
    def closeEvent(self, event):
        self.attendanceThread.go_flag = False
        self.attendanceThread.quit()
        self.global_flag[0] = True
        self.heartbeatThread.startCheck=False
        
        print("#"*60)
        print("[{}] : close attendance check".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
                
        event.accept()
###############################################################################################################


###############################################################################################################
class registerThread(QThread) :
    def __init__(self, lecture_id):
        QThread.__init__(self)
        self.imgs = []
        self.go_flag = True
        self.start_register = False
        self.coord_lst = []
        self.detected_lst = []
        self.confidence_lst = []
        self.lecture_id = lecture_id
        
        self.img_count = 0
        self.face_count = 0
        self.eye_count = 0
        self.count = []
        
        self.step = [1]
        self.start_saving = False
        self.finish_saving = [False]
        
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        
    def __del__(self):
        self.wait()
        self.go_flag = False
        
    def setParams(self, params) :
        self.video_capture = params[0]
        self.face_cascade = params[1]
        self.eyes_cascade = params[2]
        self.interval = params[3]
        self.global_flag = params[4]
        
    def setDialog(self, dialog) :
        self.dialog = dialog
        
    def setToken(self, token) :
        self.token = token
        
    def setImageLabel(self, imageLabel) :
        self.imageLabel = imageLabel
        self.imageLabel.setAlignment(Qt.AlignCenter)
        self.imageLabel.setScaledContents(True)
        self.imageLabel.setMinimumSize(1,1)

    def run(self):
        self.face_register()
    
    def face_register(self) :
        start_time = 0
        font = cv2.FONT_HERSHEY_PLAIN
        color = (0,0,0)
        stroke = 2
        finish_register = False
       
        while self.go_flag :
            if self.start_register :
                read, frame = self.video_capture.read()
                frame = cv2.resize(frame, WH)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                h, w, c = HWC
                end_time = time.time()

                # step1 : 고정된 사각형 생성
                if self.step[0] == 1 :
                    self.step[0] += 1
                    self.x = 165
                    self.y = 105
                    self.w = 150
                    self.h = 150
                    start_time = time.time()
                    print("[{}] : register - step1 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                    print("[{}] : register - step2 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        
                # step2 : 얼굴이 인식되는 장소에 bounding box를 그리고 얼굴을 인식하여 사진 모으기(50~100)
                # 요청  : 정면의 카메라를 보고 있어주세요. (촬영횟수 count)
                #         만약 사각형이 얼굴을 찾지 못한다면, 다시 시작해주세요.
                elif self.step[0] == 2 :                       
                    flag, detected_frame, coords, eye = self.detect(gray, frame, (0,0,255))
                    self.img_count += 1
                    
                    if len(coords) != 0 :
                        self.face_count += 1
                        area, check = self.check_overlap(coords[0])
                    else :
                        area = 0
                        check = False
                    
                    # 5초 뒤에 시작
                    if end_time - start_time >= 5 :
                        name = "mathcing 80% rectangle,    complete {:.1f}%".format(len(self.detected_lst)/100*100)
                        
                        if check :
                            detected_frame = cv2.cvtColor(detected_frame, cv2.COLOR_RGB2BGR)
                            self.detected_lst.append(detected_frame)
                        
                            if eye : 
                                self.eye_count += 1
                            
                        else :
                            name = "not overlap"
                    else :
                        name = "after {} seconds, start attendance check".format(int(5-(end_time - start_time)))

                    self.dialog.msg.setText(name)
                    cv2.rectangle(frame, (self.x, self.y), (self.x+self.w, self.y+self.h), (255,0,0), 2)

                    if self.img_count == 50 :
                        self.count.append((str(self.img_count), str(self.face_count), str(self.eye_count)))
                        self.img_count = 0
                        self.face_count = 0
                        self.eye_count = 0
                    
                    if len(self.detected_lst) == 100 :
                        print("[{}] : register - step2 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        print("[{}] : register - step3 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        self.step[0] += 1

                # step3 : 저장
                elif self.step[0] == 3 :
                    if not self.start_saving :
                        tThread = trainThread(self.detected_lst, self.count)
                        tThread.setStep(self.step)
                        tThread.setFlag(self.finish_saving)
                        tThread.setRecognizer(self.recognizer)
                        tThread.setCoord(self.coord_lst)
                        tThread.setToken(self.token)
                        tThread.start()
                        self.start_saving = True
                        
                    if self.finish_saving[0] :
                        self.step[0] += 1
                        start_time = time.time()
                        self.recognizer.read("./data/register/registered_info/train1.yml")
                        
                        print("[{}] : register - step3 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        print("[{}] : register - step4 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        
                    name = "saving register result... wait please"
                    self.dialog.msg.setText(name)
                    cv2.rectangle(frame, (self.x, self.y), (self.x+self.w, self.y+self.h), (255,0,0), 2)
                    
                # step4 : confidence 확인
                elif self.step[0] == 4 : 
                    flag, detected_frame, coords, eye = self.detect(gray, frame, (0,0,255))                    
                    
                    if len(coords) != 0 :
                        coord = coords[0]
                        area, check = self.check_overlap(coord)
                    else :
                        area = 0
                        check = False
                    
                    
                    # 5초 뒤에 시작
                    if end_time - start_time >= 5 :
                        name = "mathcing 80% rectangle,    complete {:.1f}%".format(len(self.confidence_lst)/100*100)
                        
                        if check :
                            roi_gray = gray[coord[1]:coord[1]+coord[3], coord[0]:coord[0]+coord[2]]
                            label, confidence = self.recognizer.predict(roi_gray)
                            self.confidence_lst.append(confidence)
                        
                            if eye : 
                                self.eye_count += 1
                            
                        else :
                            name = "not overlap"
                    else :
                        name = "after {} seconds, start attendance check".format(int(5-(end_time - start_time)))

                    self.dialog.msg.setText(name)
                    cv2.rectangle(frame, (self.x, self.y), (self.x+self.w, self.y+self.h), (255,0,0), 2)

                    if len(self.confidence_lst) == 100 :
                        cThread = confidenceThread(self.confidence_lst)
                        cThread.start()
                        self.start_saving = True
                        
                        print("[{}] : register - step4 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        print("")
                        
                        self.start_register = False
                        self.dialog.start = False
                        finish_register = True
                    
            else :
                self.step = [1]
                read, frame = self.video_capture.read()
                frame = cv2.resize(frame, WH)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                h, w, c = HWC 
            
                name = "ready for register,place at center of sreen"
                if finish_register :
                    name = "register finished"
                    
                self.dialog.msg.setText(name)
                cv2.circle(frame, (w//2, h//2), 3, (255,0,0), -1)

            q_img = QImage(frame, w, h, w*c, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_img)
            pixmap_image = QPixmap(pixmap)

            self.imageLabel.setPixmap(pixmap_image)
                
    def check_include_center(self, coord, w, h) :
        if (coord[0] < (w//2) < (coord[0]+coord[2])) and  (coord[1] < (h//2) < (coord[1]+coord[3])) :
            return True
        else :
            return False

    def check_overlap(self, coord) :
        r1 = (self.x , self.y-self.h, self.x+self.w, self.y)
        r2 = (coord[0] , coord[1]-coord[3], coord[0]+coord[2], coord[1])
        
        dx = min(r1[2], r2[2]) - max(r1[0], r2[0])
        dy = min(r1[3], r2[3]) - max(r1[1], r2[1])
        area = dx*dy
        
        if (dx>=0) and (dy>=0) and (area/(self.w*self.h) >= 0.8) :
            return area/self.w*self.h, True
        else :
            if dx*dy<= 0 :
                area = 0
            return 0, False 
                
    def detect(self, gray, frame, color):
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        face = self.size_filter(faces)
        roi_color = []
        eye = False
        
        for (x,y,w,h) in face : 
            cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)

            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            
            eyes = self.eyes_cascade.detectMultiScale(roi_gray, 1.1, 3)
            if len(eyes) == 0:
                eye = False
            else :
                eye = True
                    
        if len(face) == 0 :
            flag = False
        else :
            flag = True
            
        coord = face
        return flag, roi_color, coord , eye
        
    def set_rectangle(self, coord_lst) :
        x_lst = [coord[0] for coord in coord_lst]
        y_lst = [coord[1] for coord in coord_lst]
        w_lst = [coord[2] for coord in coord_lst]
        h_lst = [coord[3] for coord in coord_lst]

        x = int(sum(x_lst)/len(x_lst))
        y = int(sum(y_lst)/len(y_lst))
        w = int(sum(w_lst)/len(w_lst))
        h = int(sum(h_lst)/len(h_lst))
        
        self.x = x
        self.y = y
        self.w = w
        self.h = h 
        
    def size_filter(self, coords) :
        if len(coords) == 0 :
            return []
        
        size = 0 
        for coord in coords :
            now = coord[2]*coord[3]
            if now > size :
                size = now
                best_coord = coord

        return [best_coord]
    
class trainThread(QThread) :
    def __init__(self, imgs, count):
        QThread.__init__(self)
        self.imgs = imgs
        self.count = count
        
    def __del__(self):
        self.wait()

    def run(self):
        self.save_and_train()
        
    def setRecognizer(self, recognizer) :
        self.recognizer = recognizer
        
    def setStep(self, step) :
        self.step = step
    
    def setFlag(self, flag) :
        self.finish_saving = flag
    
    def setCoord(self, coord) :
        self.coord = coord
    
    def setToken(self, token) :
        self.token = token
        
    def text_rectangle(self, coords) :
        result = []
        for lst in coords :
            result.append(", ".join([str(x) for x in lst]))
            
        return result
        
    def save_and_train(self) :
        for idx, img in enumerate(self.imgs) :
            cv2.imwrite("./data/register/registered_png/register_{}.png".format(idx), img, [])
        
        labels = self.make_labels(self.imgs)
        grays = self.making_gray(self.imgs)
        
        self.recognizer.train(grays[10:], labels[10:])
        self.recognizer.save("./data/register/registered_info/train1.yml")
    
        with open('./data/register/registered_info/train1.yml', 'rb') as image_file :
            files = {'model': image_file}
            header = {"x-access-token" : self.token}
            result = requests.post(YML_IP, files=files, headers=header)
            
        coord = self.text_rectangle(self.coord)
        with open("./data/register/registered_info/rectangle_coord.txt", "w") as f:
            f.write("\n".join(coord))
            
        cnt = [", ".join(x) for x in self.count]
        with open("./data/register/registered_info/count.txt", "w") as f:
            f.write("\n".join(cnt))
        
        self.finish_saving[0] = True
        self.quit()
        
    def make_labels(self, images) :
        labels = []

        length = len(images)
        for idx in range(length) :
            labels.append(1)

        return np.array(labels)
        
    def making_gray(self, images) :
        result = []

        for image in images :
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            result.append(gray)

        return result
    
class confidenceThread(QThread) :
    def __init__(self, confidence):
        QThread.__init__(self)
        self.confidence = confidence
        
    def __del__(self):
        self.wait()

    def run(self):
        self.save()
        
    def save(self) :
        with open("./data/register/registered_info/confidence.txt", "w") as f:
            f.write("\n".join([str(conf) for conf in self.confidence]))
            
        with open("./data/register/registered_info/train_datetime.txt", "w") as f:
            f.write(str(time.strftime("%Y/%m/%d %H:%M", time.localtime(time.time()))))

        print("-"*90)
        print("[{}] : train and saving are finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("-"*90)
        print("")
        
        self.quit()
###############################################################################################################


###############################################################################################################
class attendanceThread(QThread) :
    def __init__(self, lecture_id, token):
        QThread.__init__(self)
        self.imgs = []
        self.go_flag = True
        self.start_register = False
        self.coord_lst = []
        self.confidence_lst = []
        self.lecture_id = lecture_id
        self.token = token
        
        self.step = [1]
        self.start_saving = False
        self.finish_saving = [False]
        
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()
        
    def __del__(self):
        self.wait()
        self.go_flag = False
        
    def setParams(self, params) :
        self.video_capture = params[0]
        self.face_cascade = params[1]
        self.eyes_cascade = params[2]
        self.interval = params[3]
        self.global_flag = params[4]
        
    def setDialog(self, dialog) :
        self.dialog = dialog
        
    def setImageLabel(self, imageLabel) :
        self.imageLabel = imageLabel
        self.imageLabel.setAlignment(Qt.AlignCenter)
        self.imageLabel.setScaledContents(True)
        self.imageLabel.setMinimumSize(1,1)

    def run(self):
        self.face_register()
    
    def face_register(self) :
        start_time = 0
        font = cv2.FONT_HERSHEY_PLAIN
        color = (0,0,0)
        stroke = 2
        finish_attendance = False
       
        while self.go_flag :
            if self.start_register :
                read, frame = self.video_capture.read()
                frame = cv2.resize(frame, WH)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                h, w, c = HWC
                end_time = time.time()

                # step1 : 고정된 사각형 생성
                if self.step[0] == 1 :
                    self.step[0] += 1
                    self.x = 165
                    self.y = 105
                    self.w = 150
                    self.h = 150
                    print("[{}] : attendance check - step1 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))

                # step2 : pass
                elif self.step[0] == 2 :      
                    self.step[0] += 1
                    print("[{}] : attendance check - step2 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                    print("[{}] : attendance check - step2 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))


                # step3 : pass
                elif self.step[0] == 3 :
                    self.recognizer.read("./data/register/registered_info/train1.yml")
                    self.step[0] += 1
                    start_time = time.time()
                    print("[{}] : attendance check - step3 start".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                    print("[{}] : attendance check - step3 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))

                    
                # step4 : 서버에 결과 전송
                elif self.step[0] == 4 : 
                    flag, detected_frame, coords, eye = self.detect(gray, frame, (0,0,255))                    
                    
                    if len(coords) != 0 :
                        coord = coords[0]
                        area, check = self.check_overlap(coord)
                    else :
                        area = 0
                        check = False
                    
                    if end_time - start_time >= 5 :
                        name = "mathcing 80% rectangle,    complete {:.1f}%".format(len(self.confidence_lst)/100*100)
                        
                        if check :
                            roi_gray = gray[coord[1]:coord[1]+coord[3], coord[0]:coord[0]+coord[2]]
                            label, confidence = self.recognizer.predict(roi_gray)
                            self.confidence_lst.append(confidence)
                            self.coord_lst.append(coord)
                            
                        else :
                            name = "not overlap"
                    else :
                        name = "after {} seconds, start attendance check".format(int(5-(end_time - start_time)))
                        
                    self.dialog.msg.setText(name)
                    cv2.rectangle(frame, (self.x, self.y), (self.x+self.w, self.y+self.h), (255,0,0), 2)

                    if len(self.confidence_lst) == 100 :
                        cThread = confidenceThread2(self.confidence_lst, self.coord_lst, self.lecture_id, self.token)
                        
                        cThread.start()
                        self.start_saving = True
                        
                        print("[{}] : attendance check - step4 finished".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
                        print("")
                        
                        self.start_register = False
                        self.dialog.start = False
                        finish_attendance = True
                    
            else :
                self.step = [1]
                read, frame = self.video_capture.read()
                frame = cv2.resize(frame, WH)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                h, w, c = HWC 
            
                name = "ready for register,place at center of sreen"
                if finish_attendance :
                    name = "attendance check finished"
                    
                self.dialog.msg.setText(name)
                cv2.circle(frame, (w//2, h//2), 3, (255,0,0), -1)

            q_img = QImage(frame, w, h, w*c, QImage.Format_RGB888)
            pixmap = QPixmap.fromImage(q_img)
            pixmap_image = QPixmap(pixmap)

            self.imageLabel.setPixmap(pixmap_image)
                
    def check_include_center(self, coord, w, h) :
        if (coord[0] < (w//2) < (coord[0]+coord[2])) and  (coord[1] < (h//2) < (coord[1]+coord[3])) :
            return True
        else :
            return False

    def check_overlap(self, coord) :
        r1 = (self.x , self.y-self.h, self.x+self.w, self.y)
        r2 = (coord[0] , coord[1]-coord[3], coord[0]+coord[2], coord[1])
        
        dx = min(r1[2], r2[2]) - max(r1[0], r2[0])
        dy = min(r1[3], r2[3]) - max(r1[1], r2[1])
        area = dx*dy
        
        if (dx>=0) and (dy>=0) and (area/(self.w*self.h) >= 0.8) :
            return area/self.w*self.h, True
        else :
            if dx*dy<= 0 :
                area = 0
            return 0, False 
                
    def detect(self, gray, frame, color):
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        face = self.size_filter(faces)
        roi_color = []
        eye = False
        
        for (x,y,w,h) in face : 
            cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)

            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            
            eyes = self.eyes_cascade.detectMultiScale(roi_gray, 1.1, 3)
            if len(eyes) == 0:
                eye = False
            else :
                eye = True
                    
        if len(face) == 0 :
            flag = False
        else :
            flag = True
            
        coord = face
        return flag, roi_color, coord , eye
        
    def set_rectangle(self, coord_lst) :
        x_lst = [coord[0] for coord in coord_lst]
        y_lst = [coord[1] for coord in coord_lst]
        w_lst = [coord[2] for coord in coord_lst]
        h_lst = [coord[3] for coord in coord_lst]

        x = int(sum(x_lst)/len(x_lst))
        y = int(sum(y_lst)/len(y_lst))
        w = int(sum(w_lst)/len(w_lst))
        h = int(sum(h_lst)/len(h_lst))
        
        self.x = x
        self.y = y
        self.w = w
        self.h = h 
        
    def size_filter(self, coords) :
        if len(coords) == 0 :
            return []
        
        size = 0 
        for coord in coords :
            now = coord[2]*coord[3]
            if now > size :
                size = now
                best_coord = coord

        return [best_coord]
    
class confidenceThread2(QThread) :
    def __init__(self, confidence, coord, lecture_id, token):
        QThread.__init__(self)
        self.confidence = confidence
        self.coord = coord
        self.lecture_id = lecture_id
        self.token = token
        
    def __del__(self):
        self.wait()

    def run(self):
        self.save()
        
    def mean(self, lst) :
        result = 0
        for l in lst :
            result+= l
        return result/len(lst)
        
    def save(self) :
        
        with open("./data/detect/detected_info/attendance.txt", "w") as f:
            for confidence, coord in zip(self.confidence, self.coord) :
                f.write(str(confidence)+", "+str(coord[0])+", "+str(coord[1])+", "+str(coord[2])+", "+str(coord[3])+", "+str(self.lecture_id ))
                f.write("\n")
        
        conf_mean = self.mean(self.confidence)
        x_mean = self.mean([x[0] for x in self.coord])
        y_mean = self.mean([x[1] for x in self.coord])
        w_mean = self.mean([x[2] for x in self.coord])
        h_mean = self.mean([x[3] for x in self.coord])
        data = {"lecture_id": self.lecture_id,
                "x_position": x_mean,
                "y_position":y_mean,
                "frame_width":w_mean,
                "frame_height":h_mean,
                "confidence":conf_mean}

        header = {"x-access-token" : self.token}
        result = requests.post(ATTENDANCE_IP , data=data, headers = header)

        print("-"*90)
        print("[{}] : attendance check is sent".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("-"*90)
        print("")
        
        self.quit()

In [9]:
class mainWindow(QMainWindow) :
    def __init__(self) :
        QMainWindow.__init__(self)
        self.setWindowTitle("PyQt & openCV demo3")
        self.resize(300,200)
        
        self.main_widget = BeforeLogin()
        self.setCentralWidget(self.main_widget)
        
    def closeEvent(self, event) :
        reply = QMessageBox.question(self, 'Message',"Are you sure to quit?", 
                                           QMessageBox.Yes, QMessageBox.No)

        if reply == QMessageBox.Yes:
            if self.main_widget.token :
                self.start = False
                
                try :
                    self.main_widget.lectureWidget.afterLoginWidget.registerDialog.close()
                except :
                    pass
                
                try :
                    self.main_widget.lectureWidget.afterLoginWidget.attendanceDialog.close()
                except :
                    pass
                
                try :
                    self.main_widget.lectureWidget.afterLoginWidget.video_capture.release()
                except :
                    pass
                
                try :
                    self.main_widget.lectureWidget.afterLoginWidget.heartbeatThread.state=False
                except :
                    pass
                
            event.accept()
        else:
            event.ignore()

        
        event.accept()
        
def main():
    app = QApplication([])
    main_window = mainWindow()
    main_window.show()
    
    try :
        sys.exit(app.exec_())
        
        print("#"*60)
        print("[{}] : (1) program stop".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
        
    except :
        print("")
        print("#"*60)
        print("[{}] : (2) program stop".format(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(time.time()))))
        print("#"*60, "\n")
        
if __name__ == '__main__':
    main()

############################################################
[2019/02/27 13:55:52] : program start
############################################################ 


############################################################
[2019/02/27 13:55:55] : (2) program stop
############################################################ 

